In [ ]:
!pip install setuptools==66
!pip install matplotlib_inline
!pip install d2l==1.0.0b
!pip install pytorch-pretrained-bert

### Use BERT for review classification
Apply these 2 for experiments:

1. Use pretrain BERT
2. Train BERT from scratch and use it

Use Case: 
1. Binary classification ( good or bad)
2. Multi-class classification ( good or bad)


In [ ]:
import torch
from torch import nn
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
import logging 
from d2l import torch as d2l

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

100%|██████████| 231508/231508 [00:00<00:00, 3028978.96B/s]


### Pretrained BERT:
12 layer
768 hidden
12 heads
110M parameters

In [ ]:
text = "[CLS] Who was Jim Henson ? [SEP] Jim Henson was a puppeteer [SEP]"
tokens = tokenizer.tokenize(text)
indexed_tokens = tokenizer.convert_tokens_to_ids(tokens)
print(tokens)
print(indexed_tokens)
segments_ids = [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]

tokens = torch.tensor([indexed_tokens])
segments = torch.tensor([segments_ids])

model = BertModel.from_pretrained('bert-base-uncased')
model.eval()

#convert to cuda device
tokens = tokens.to('cuda')
segments = segments.to('cuda')
model.to('cuda')

with torch.no_grad():
  encoded_layers, _ = model(tokens, segments)

print(len(encoded_layers))

['[CLS]', 'who', 'was', 'jim', 'henson', '?', '[SEP]', 'jim', 'henson', 'was', 'a', 'puppet', '##eer', '[SEP]']
[101, 2040, 2001, 3958, 27227, 1029, 102, 3958, 27227, 2001, 1037, 13997, 11510, 102]


 58%|█████▊    | 237019136/407873900 [00:04<00:04, 40455831.04B/s]

### Sentiment Analysis
1. with RNN using Glove

In [ ]:
class BiRNN(nn.Module):
  def __init__(self, vocab_size, embed_size, num_hiddens, num_layers, **kwargs):
    super(BiRNN, self).__init__(**kwargs)

    # self.embedding = BertModel.from_pretrained('bert-base-uncased')
    self.embedding = nn.Embedding(vocab_size, embed_size)
    self.encoder = nn.LSTM(embed_size, num_hiddens, num_layers = num_layers, bidrectional=True)

    self.decoder = nn.Linear(4 * num_hiddens, 2)

  def forward(self, inputs):
    # original inputs = (# batch, # steps)
    #transform inputs to (# steps, # batch)
    embedding = self.embedding(inputs.T)
    self.encoder.flatten_parameters()
    #output shape = (# steps, # batch, # word vector dim)
    outputs, _ = self.encoder(embeddings)

    print('output shape = ', outputs.shape)

    encoding = torch.cat((outputs[0], outputs[-1]), dim = 1)
    print('encoding shape after concat = ',encoding.shape)

    outs = self.decoder(encoding) 

    return out
  





In [ ]:
#Test run 

In [ ]:
#load glove
glove_embedding = d2l.TokenEmbedding('glove.6b.100d')
embeds = glove_embedding[vocab.idx_to_token]
print(embeds.shape)



